In [1]:
# sys.path를 수정하여 모듈 경로를 설정
import sys
import os
import pandas as pd

# 프로젝트 루트 경로 추가
project_root = os.path.abspath("..")
sys.path.append(project_root)


# 모든 컬럼 출력 설정
pd.set_option('display.max_columns', None)

# 모든 행 출력 설정
pd.set_option('display.max_rows', None)
import module


In [2]:
df = module.file_load.load_current_version()

In [3]:
module.preprocess.시_군_구_add(df)
module.parser.아파트_unique_key_add(df)

In [4]:
df.drop_duplicates(subset='아파트_unique_key', keep='first', inplace=True)

In [5]:
api_cache = pd.read_csv('api_카테고리검색_cache.csv')
api_cache = api_cache.set_index('query')
def find_cash_and_save(query):
    if query in api_cache.index:
        print('cache hit')
        return api_cache.loc[query]['result']
    else:
        result = module.api.장소검색(query)
        api_cache.loc[query] = result
        return result


In [6]:
df_select = df[
['시군구', '번지', '본번', '부번', '아파트명',
        '건축년도', '도로명', 
        '좌표X', '좌표Y',
        '구', '동', '아파트_unique_key']        
]

In [34]:
# 편의점 500, 학원 500, 학교 500, 공공기관 500, 병원 500, 음식점, "어린이집, 유치원"
df_select['편의점'] = 0
df_select['학원'] = 0
df_select['학교'] = 0
df_select['공공기관'] = 0
df_select['병원'] = 0
df_select['음식점'] = 0
df_select['어린이집, 유치원'] = 0



/tmp/ipykernel_3226321/2073180525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['편의점'] = 0
/tmp/ipykernel_3226321/2073180525.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['학원'] = 0
/tmp/ipykernel_3226321/2073180525.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [74]:
# # 편의점 500, 학원 500, 학교 500, 공공기관 500, 병원 500, 음식점, "어린이집, 유치원"

category = "어린이집, 유치원"

In [75]:
print(category)
# 편의점 , 학교, 학원, 음식점, 병원
# query = module.api.장소검색_쿼리(127.060972, 37.500979, 300, "편의점")

# result = find_cash_and_save(query)
# query, result
cnt = 0
for index, row in df_select.iterrows():
    query = module.api.장소검색_쿼리(row['좌표X'], row['좌표Y'], 500, category)
    result = find_cash_and_save(query)
    df_select.loc[index, category] = result
    cnt += 1
    if cnt % 500 == 0:
        print(cnt)


어린이집, 유치원
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
500
cache hit
cache hit
cache hit
1000
cache hit
cache hit
1500
cache hit
cache hit
2000
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
2500
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
3000
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
3500
cache hit
4000
cache hit
cache hit
4500
cache hit
cache hit
cache hit
cache hit
cache hit
5000
cache hit
5500
6000
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
6500
cache hit
cache hit
cache hit
cache hit
cache hit
7000
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
7500
cache hit
cache hit
cache hit
cache hit
cache hit
8000
cache hit
8500
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache hit
cache 

In [76]:
print(df_select[category].isnull().sum())
print(df_select[df_select[category] == 0].shape)

0
(47, 19)


In [77]:
df_select[category].value_counts().sort_index()

어린이집, 유치원
0      47
1     113
2     228
3     376
4     464
5     586
6     644
7     603
8     582
9     710
10    634
11    568
12    590
13    500
14    397
15    320
16    286
17    256
18    235
19    208
20    210
21    181
22    139
23     90
24     24
25     11
26     12
27      7
28      6
29      2
30      1
31      1
35      1
Name: count, dtype: int64

In [78]:
df_select.to_csv('주변시설_apt_편의점_학원_학교_공공기관_병원_음식점_어린이집.csv', index=False)
df_select.to_csv('주변시설_apt.csv', index=False)

In [79]:
api_cache.to_csv('api_카테고리검색_cache.csv')

: 

In [ ]:
df.columns